In [17]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

In [18]:
df = pd.read_csv("/Users/ethanmathew/Downloads/ShowcaseDataWinter2023.csv")
df

,price,appraisal_offer,online_appraisal_flag,model_year,mileage,make,model,trim_descrip,body,color,...,trim_descrip_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,$20k to $25k,$0k to $5k,1.0,2014.0,15k to 20k miles,HDT,HDT_12,Premium,E,Red,...,NaN,D,Silver,4.0L,6,17.0,21,236,21.0,13
1,$15k to $20k,$0k to $5k,0.0,2006.0,100k+ miles,KQZ,KQZ_23,Premium,E,Silver,...,Not Premium,D,Gold,4.2L,6,16.0,21,205,25.0,11
2,$30k to $35k,$35k to $40k,1.0,2014.0,5k to 10k miles,CZV,CZV_10,Premium,E,Black,...,NaN,E,Red,2.0L,4,41.0,38,181,10.0,13
3,$35k to $40k,$5k to $10k,1.0,2014.0,15k to 20k miles,IZA,IZA_19,Premium,F,Silver,...,Premium,F,Blue,3.0L,6,17.0,25,230,16.0,13
4,$30k to $35k,$15k to $20k,1.0,2011.0,35k to 40k miles,HDT,HDT_8,Premium,F,Blue,...,Not Premium,E,Black,2.4L,4,20.0,29,182,18.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,$15k to $20k,$15k to $20k,1.0,2013.0,10k to 15k miles,OFW,OFW_6,Premium,F,Black,...,Not Premium,E,Black,1.4L,4,26.0,31,138,14.0,1
199996,$20k to $25k,$10k to $15k,0.0,2014.0,55k to 60k miles,ARU,ARU_12,Not Premium,F,White,...,Not Premium,B,Black,1.8L,4,30.0,39,143,13.0,8
199997,$30k to $35k,$0k to $5k,0.0,2014.0,10k to 15k miles,AIH,AIH_13,Premium,F,Silver,...,Not Premium,F,Gray,2.4L,4,21.0,30,177,18.0,3
199998,$30k to $35k,$10k to $15k,0.0,2012.0,50k to 55k miles,KQZ,KQZ_3,Not Premium,D,Black,...,Premium,D,White,5.7L,8,13.0,19,390,26.0,11


## Cleaning Data To Get Desired Inputs

In [19]:
def drop_variables(df):
    df.drop(["online_appraisal_flag", "model_year", "mileage", 'make', "model", 'trim_descrip', 'body', 'color', 'engine', 'cylinders', 'mpg_city', 'mpg_highway', 'horsepower', 'fuel_capacity', 'model_year_appraisal', "trim_descrip_appraisal", "make_appraisal"], axis = 1, inplace = True)
    df.drop(df[df.isna().any(axis=1)].index, inplace=True, axis = 0)
    return df
drop_variables(df)


,price,appraisal_offer,mileage_appraisal,model_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,$20k to $25k,$0k to $5k,200k+ miles,AIH_32,D,Silver,4.0L,6,17.0,21,236,21.0,13
1,$15k to $20k,$0k to $5k,0 to 10k miles,KQZ_3,D,Gold,4.2L,6,16.0,21,205,25.0,11
2,$30k to $35k,$35k to $40k,10k to 20k miles,CIP_16,E,Red,2.0L,4,41.0,38,181,10.0,13
4,$30k to $35k,$15k to $20k,20k to 30k miles,LTX_20,E,Black,2.4L,4,20.0,29,182,18.0,16
5,$15k to $20k,$5k to $10k,160k to 170k miles,FJE_10,F,Black,3.6L,6,18.0,27,292,19.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,$15k to $20k,$15k to $20k,0 to 10k miles,LTX_30,E,Black,1.4L,4,26.0,31,138,14.0,1
199996,$20k to $25k,$10k to $15k,50k to 60k miles,HXQ_8,B,Black,1.8L,4,30.0,39,143,13.0,8
199997,$30k to $35k,$0k to $5k,120k to 130k miles,HXQ_14,F,Gray,2.4L,4,21.0,30,177,18.0,3
199998,$30k to $35k,$10k to $15k,130k to 140k miles,BZQ_4,D,White,5.7L,8,13.0,19,390,26.0,11


In [20]:
print(df["appraisal_offer"].unique())
print(df["price"].unique())

['$0k to $5k' '$35k to $40k' '$15k to $20k' '$5k to $10k' '$40k+'
 '$20k to $25k' '$10k to $15k' '$30k to $35k' '$25k to $30k']
['$20k to $25k' '$15k to $20k' '$30k to $35k' '$35k to $40k'
 '$45k to $50k' '$25k to $30k' '$40k to $45k' '$0 to $15k' '$60k to $65k'
 '$65k to $70k' '$55k to $60k' '$70k+' '$50k to $55k']


In [21]:
def get_offer(offer):
    if(offer.find("to") > 0):
        offer = offer.split('to')[0].strip()
        offer = offer[1:len(offer) -1]
        offer = int(offer)
        return offer
    else:
        offer = offer[1:len(offer)-2]
        offer = int(offer)
        return offer
df["appraisal_offer"] = df["appraisal_offer"].apply(lambda x: get_offer(x))

In [22]:
def get_engine(engine):
    return engine[:len(engine)-1]
df["engine_appraisal"] = df["engine_appraisal"].apply(lambda x: get_engine(x))

In [23]:
def get_mileage(mileage):
    if(mileage.find("to") > 0):
        mileage = mileage.split('to')[0].strip()
        if(len(mileage) > 1):
            mileage = mileage[0:len(mileage) -1]
        mileage = int(mileage)
        return mileage
    else:
        mileage = mileage[0:len(mileage)-8]
        mileage = int(mileage)
        return mileage
df["mileage_appraisal"]= df["mileage_appraisal"].apply(lambda x: get_mileage(x))

In [24]:
df['appraisal_offer'] = pd.to_numeric(df['appraisal_offer'])
df['mileage_appraisal'] = pd.to_numeric(df['mileage_appraisal'])
df['engine_appraisal'] = pd.to_numeric(df['engine_appraisal'])
df['cylinders_appraisal'] = pd.to_numeric(df['cylinders_appraisal'])
df['mpg_city_appraisal'] = pd.to_numeric(df['mpg_city_appraisal'])
df['mpg_highway_appraisal'] = pd.to_numeric(df['mpg_highway_appraisal'])
df['horsepower_appraisal'] = pd.to_numeric(df['horsepower_appraisal'])
df['fuel_capacity_appraisal'] = pd.to_numeric(df['fuel_capacity_appraisal'])
df['market'] = pd.to_numeric(df['market'])

In [25]:
df.head()

,price,appraisal_offer,mileage_appraisal,model_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,$20k to $25k,0,200,AIH_32,D,Silver,4.0,6,17.0,21,236,21.0,13
1,$15k to $20k,0,0,KQZ_3,D,Gold,4.2,6,16.0,21,205,25.0,11
2,$30k to $35k,35,10,CIP_16,E,Red,2.0,4,41.0,38,181,10.0,13
4,$30k to $35k,15,20,LTX_20,E,Black,2.4,4,20.0,29,182,18.0,16
5,$15k to $20k,5,160,FJE_10,F,Black,3.6,6,18.0,27,292,19.0,10


In [26]:
non_categorial_features = ['appraisal_offer',
                          'mileage_appraisal',
                          'cylinders_appraisal',
                          'mpg_city_appraisal',
                          'mpg_highway_appraisal', 
                          'horsepower_appraisal',
                          'fuel_capacity_appraisal',
                          'market',
                          'price', 
                          'engine_appraisal']

for categorical_feature in list(df.columns):
    if categorical_feature not in non_categorial_features:
        df[categorical_feature] = df[categorical_feature].astype('category')
        one_hot = pd.get_dummies(df[categorical_feature])
        df = df.drop(categorical_feature, axis = 1)
        df = df.join(one_hot)




In [27]:
one_hot_price = pd.get_dummies(df['price'])
df = df.drop('price', axis = 1)
df = df.join(one_hot_price)
df.head()

,appraisal_offer,mileage_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market,AIH_10,...,$25k to $30k,$30k to $35k,$35k to $40k,$40k to $45k,$45k to $50k,$50k to $55k,$55k to $60k,$60k to $65k,$65k to $70k,$70k+
0,0,200,4.0,6,17.0,21,236,21.0,13,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,4.2,6,16.0,21,205,25.0,11,0,...,0,0,0,0,0,0,0,0,0,0
2,35,10,2.0,4,41.0,38,181,10.0,13,0,...,0,1,0,0,0,0,0,0,0,0
4,15,20,2.4,4,20.0,29,182,18.0,16,0,...,0,1,0,0,0,0,0,0,0,0
5,5,160,3.6,6,18.0,27,292,19.0,10,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
def format_data(df):
    X = df.iloc[ :, 0:len(df.axes[1]) - 13].to_numpy()
    y = df.iloc[: , len(df.axes[1]) - 13:len(df.axes[1])].to_numpy()
    return X, y
X, y = format_data(df)

print ('The shape of X is: ' + str(X.shape))
print ('The shape of y is: ' + str(y.shape))


The shape of X is: (177045, 708)
The shape of y is: (177045, 13)


In [29]:
#creating training, test and cross validation data
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [30]:
# normalize the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [31]:
tf.random.set_seed(1234)
model = Sequential(
    [               
        tf.keras.Input(shape = (708,)),
        Dense(1024, activation = "relu"),
        Dense(512, activation = 'relu'),
        Dense(256, activation = 'relu'),
        Dense(13, activation = 'linear')

    ], name = "my_model" 
)
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              726016    
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 13)                3341      
                                                                 
Total params: 1,385,485
Trainable params: 1,385,485
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits= True) ,
    optimizer=tf.keras.optimizers.Adam(learning_rate=1E-5),
    metrics=["accuracy"]
)
history = model.fit(
    X_train, y_train, 
    validation_data=(X_val, y_val), 
    epochs=20)


Epoch 1/20
3320/3320 [==============================] - 22s 7ms/step - loss: 2.0507 - accuracy: 0.2409 - val_loss: 1.9475 - val_accuracy: 0.2515
Epoch 2/20
3320/3320 [==============================] - 22s 7ms/step - loss: 1.9065 - accuracy: 0.2637 - val_loss: 1.8966 - val_accuracy: 0.2588
Epoch 3/20
3320/3320 [==============================] - 22s 7ms/step - loss: 1.8669 - accuracy: 0.2710 - val_loss: 1.8816 - val_accuracy: 0.2613
Epoch 4/20
3320/3320 [==============================] - 22s 7ms/step - loss: 1.8479 - accuracy: 0.2745 - val_loss: 1.8755 - val_accuracy: 0.2609
Epoch 5/20
3320/3320 [==============================] - 21s 6ms/step - loss: 1.8366 - accuracy: 0.2780 - val_loss: 1.8729 - val_accuracy: 0.2597
Epoch 6/20
3320/3320 [==============================] - 21s 6ms/step - loss: 1.8292 - accuracy: 0.2794 - val_loss: 1.8705 - val_accuracy: 0.2610
Epoch 7/20
3320/3320 [==============================] - 21s 6ms/step - loss: 1.8232 - accuracy: 0.2827 - val_loss: 1.8705 - val_ac

In [ ]:
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [21]:
#accuracy is really really bad but I can change the output and then balance the data
y_prediction = (model.predict(X_test)).astype(np.float32)
print(y_prediction.size == y_test.size)

for i in range(30):
    print( f"{np.argmax(y_test[i])}, category: {np.argmax(y_prediction[i])}")


1107/1107 [==============================] - 3s 3ms/step
True
3, category: 1
1, category: 2
5, category: 2
5, category: 1
3, category: 2
3, category: 3
4, category: 3
1, category: 3
0, category: 2
5, category: 3
3, category: 3
11, category: 3
3, category: 3
1, category: 2
0, category: 1
2, category: 4
2, category: 3
2, category: 1
5, category: 5
3, category: 1
2, category: 2
1, category: 3
1, category: 3
2, category: 3
4, category: 3
1, category: 1
0, category: 2
5, category: 3
1, category: 2
5, category: 4


In [254]:
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(y_prediction, y_test)
m.result().numpy()

0.26575166

In [262]:
for columns in one_hot_price.columns:
    print (f"{columns} : {100 * df[columns].eq(1).mean()}")


$0 to $15k : 4.50732864525968
$15k to $20k : 17.652574204298343
$20k to $25k : 23.563500804880118
$25k to $30k : 20.18582846169053
$30k to $35k : 13.94391256460222
$35k to $40k : 9.795814623400831
$40k to $45k : 5.195289333220368
$45k to $50k : 2.6597757632240393
$50k to $55k : 1.1787963512101443
$55k to $60k : 0.6246999350447626
$60k to $65k : 0.2959699511423649
$65k to $70k : 0.1671891327063741
$70k+ : 0.2293202293202293


In [256]:

def balance_date(price_list):  
    dataframe_collection = {} 
    sizes = {}
    for prices in price_list:
        dataframe_collection[prices] = df[df[prices] == 1.0]
        df1 = df[df[prices] == 1.0]
        sizes[prices] = len(df1)
    size = min(sizes.values())
    balanced_data = pd.DataFrame()
    for prices in dataframe_collection.values():
        x = prices.sample(size)
        balanced_data = pd.concat([balanced_data, x])
    return balanced_data
balanced_data = balance_date(one_hot_price)


In [257]:
X_balanced, y_balanced = format_data(balanced_data)

X_btrain, X_btest, y_btrain, y_btest  = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=1)
X_btrain, X_bval, y_btrain, y_bval  = train_test_split(X_btrain, y_btrain, test_size=0.25, random_state=1)

scaler = StandardScaler().fit(X_btrain)
X_btrain = scaler.transform(X_btrain)
X_bval = scaler.transform(X_bval)
X_btest = scaler.transform(X_btest)

In [258]:
# this overfits like crazy 
historyb = model.fit(
    X_btrain, y_btrain, 
    validation_data=(X_bval, y_bval), 
    epochs=20)

Epoch 1/20
73/73 [==============================] - 0s 4ms/step - loss: 2.5113 - accuracy: 0.1300 - val_loss: 2.2812 - val_accuracy: 0.1597
Epoch 2/20
73/73 [==============================] - 0s 4ms/step - loss: 2.2314 - accuracy: 0.1863 - val_loss: 2.3008 - val_accuracy: 0.1610
Epoch 3/20
73/73 [==============================] - 0s 4ms/step - loss: 2.1515 - accuracy: 0.1976 - val_loss: 2.2792 - val_accuracy: 0.1558
Epoch 4/20
73/73 [==============================] - 0s 4ms/step - loss: 2.0983 - accuracy: 0.2205 - val_loss: 2.3288 - val_accuracy: 0.1545
Epoch 5/20
73/73 [==============================] - 0s 4ms/step - loss: 2.0552 - accuracy: 0.2244 - val_loss: 2.3236 - val_accuracy: 0.1571
Epoch 6/20
73/73 [==============================] - 0s 4ms/step - loss: 2.0107 - accuracy: 0.2400 - val_loss: 2.3659 - val_accuracy: 0.1740
Epoch 7/20
73/73 [==============================] - 0s 4ms/step - loss: 1.9732 - accuracy: 0.2517 - val_loss: 2.3924 - val_accuracy: 0.1584
Epoch 8/20
73/73 [==

In [259]:
y_prediction = (model.predict(X_btest)).astype(np.float32)

for i in range(10):
    print( f"{np.argmax(y_btest[i])}, category: {np.argmax(y_prediction[i])}")

25/25 [==============================] - 0s 2ms/step
10, category: 12
0, category: 1
7, category: 6
8, category: 12
8, category: 1
10, category: 12
10, category: 9
2, category: 0
0, category: 1
5, category: 3


In [191]:
new_data = df.copy(deep = True)
new_data['price'].unique()


array(['$20k to $25k', '$15k to $20k', '$30k to $35k', '$35k to $40k',
       '$45k to $50k', '$25k to $30k', '$40k to $45k', '$0 to $15k',
       '$55k to $60k', '$60k to $65k', '$65k to $70k', '$70k+',
       '$50k to $55k'], dtype=object)

In [192]:
def get_price(offer):
    if(offer.find("to") > 0):
        offer = offer.split('to')[0].strip()
        if offer.find('k') < 0:
            offer = offer[1: len(offer)]
        else:
            offer = offer[1:len(offer) -1 ]
    else:
        offer = offer[1:len(offer)-2]
    offer = int(offer)
    return offer
new_data["price"] = new_data["price"].apply(lambda x: get_price(x))
new_data["price"].unique()




array([20, 15, 30, 35, 45, 25, 40,  0, 55, 60, 65, 70, 50])

In [154]:
def format_new_data(df):
    X = df.iloc[ :, 1:len(df.axes[1])].to_numpy()
    y = df.iloc[: , 0:1].to_numpy()
    return X, y
X, y = format_new_data(new_data)

In [155]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [156]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [163]:
tf.random.set_seed(1234)
model2 = Sequential(
    [               
        tf.keras.Input(shape = (708,)),
        Dense(512, activation = 'relu'),
        Dense(256, activation = 'relu'),
        Dense(1)

    ], name = "my_model" 
)
model2.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               363008    
                                                                 
 dense_25 (Dense)            (None, 256)               131328    
                                                                 
 dense_26 (Dense)            (None, 1)                 257       
                                                                 
Total params: 494,593
Trainable params: 494,593
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.compile(loss= tf.keras.losses.MeanAbsoluteError(),
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=tf.metrics.RSquare)
history = model.fit(
    X_train, y_train, 
    validation_data=(X_val, y_val),
    epochs=30)

AttributeError: module 'keras.api._v2.keras.metrics' has no attribute 'RSquare'

In [193]:
new_data.head()

,price,appraisal_offer,online_appraisal_flag,model_year,mileage,make,model,trim_descrip,body,color,...,trim_descrip_appraisal,body_appraisal,color_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
0,20,0,1.0,2014.0,15,HDT,HDT_12,Premium,E,Red,...,NaN,D,Silver,4.0,6,17.0,21,236,21.0,13
1,15,0,0.0,2006.0,100,KQZ,KQZ_23,Premium,E,Silver,...,Not Premium,D,Gold,4.2,6,16.0,21,205,25.0,11
2,30,35,1.0,2014.0,5,CZV,CZV_10,Premium,E,Black,...,NaN,E,Red,2.0,4,41.0,38,181,10.0,13
3,35,5,1.0,2014.0,15,IZA,IZA_19,Premium,F,Silver,...,Premium,F,Blue,3.0,6,17.0,25,230,16.0,13
4,30,15,1.0,2011.0,35,HDT,HDT_8,Premium,F,Blue,...,Not Premium,E,Black,2.4,4,20.0,29,182,18.0,16


In [194]:
new_data.corr()

/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/594412850.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_data.corr()


,price,appraisal_offer,online_appraisal_flag,model_year,mileage,cylinders,mpg_city,mpg_highway,horsepower,fuel_capacity,model_year_appraisal,mileage_appraisal,engine_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
price,1.000000,0.385880,0.041594,0.483956,-0.413605,0.523591,-0.459785,-0.542658,0.667646,0.548989,0.289803,-0.248326,0.225391,0.209488,-0.176884,-0.196492,0.343596,0.233586,0.002548
appraisal_offer,0.385880,1.000000,0.186955,0.181379,-0.132513,0.210956,-0.175686,-0.200543,0.279086,0.207118,0.727130,-0.743954,0.174880,0.150371,-0.088396,-0.121064,0.413199,0.165624,0.000702
online_appraisal_flag,0.041594,0.186955,1.000000,0.029188,0.000164,0.011281,-0.009353,-0.012310,0.019515,0.006661,0.181249,-0.175881,-0.028143,-0.032616,0.035227,0.035464,0.025474,-0.026120,-0.001327
model_year,0.483956,0.181379,0.029188,1.000000,-0.678519,-0.055073,0.091571,0.039846,0.025740,-0.053252,0.216339,-0.177306,-0.002323,-0.012534,0.011048,0.017711,0.067403,0.010283,-0.017409
mileage,-0.413605,-0.132513,0.000164,-0.678519,1.000000,0.086780,-0.089594,-0.054614,0.006137,0.107328,-0.149833,0.167366,0.015046,0.019390,-0.016524,-0.024226,-0.043345,0.012529,0.007678
cylinders,0.523591,0.210956,0.011281,-0.055073,0.086780,1.000000,-0.658312,-0.667535,0.832781,0.737832,0.116621,-0.092012,0.271466,0.257158,-0.213492,-0.229087,0.302746,0.260887,-0.001470
mpg_city,-0.459785,-0.175686,-0.009353,0.091571,-0.089594,-0.658312,1.000000,0.914054,-0.746578,-0.725787,-0.116738,0.094626,-0.218648,-0.203560,0.224248,0.225704,-0.258404,-0.222582,0.016520
mpg_highway,-0.542658,-0.200543,-0.012310,0.039846,-0.054614,-0.667535,0.914054,1.000000,-0.735475,-0.752919,-0.133058,0.113092,-0.224962,-0.208624,0.220552,0.243465,-0.264568,-0.233247,0.013287
horsepower,0.667646,0.279086,0.019515,0.025740,0.006137,0.832781,-0.746578,-0.735475,1.000000,0.764105,0.181891,-0.152154,0.286343,0.269976,-0.233612,-0.241004,0.362390,0.277965,0.001770
fuel_capacity,0.548989,0.207118,0.006661,-0.053252,0.107328,0.737832,-0.725787,-0.752919,0.764105,1.000000,0.116842,-0.083947,0.277824,0.259172,-0.234696,-0.256131,0.304387,0.299922,-0.004098


In [199]:
df1 = df[df["online_appraisal_flag"] == 1]
df2 = df[df["online_appraisal_flag"] == 0]
df1["appraisal_offer"] = df1["appraisal_offer"].apply(lambda x: get_offer(x))
df1["engine_appraisal"] = df1["engine_appraisal"].apply(lambda x: get_engine(x))
df1["mileage_appraisal"]= df1["mileage_appraisal"].apply(lambda x: get_mileage(x))
df2["appraisal_offer"] = df2["appraisal_offer"].apply(lambda x: get_offer(x))
df2["engine_appraisal"] = df2["engine_appraisal"].apply(lambda x: get_engine(x))
df2["mileage_appraisal"]= df2["mileage_appraisal"].apply(lambda x: get_mileage(x))
df1["price"] = df1["price"].apply(lambda x: get_price(x))
df2["price"] = df2["price"].apply(lambda x: get_price(x))



/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/3782109529.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["appraisal_offer"] = df1["appraisal_offer"].apply(lambda x: get_offer(x))
/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/3782109529.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["engine_appraisal"] = df1["engine_appraisal"].apply(lambda x: get_engine(x))
/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/3782109529.py:5: SettingWithCopy

In [200]:
df1.corr()

/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/473017434.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr()


,price,appraisal_offer,online_appraisal_flag,model_year,cylinders,mpg_city,mpg_highway,horsepower,fuel_capacity,model_year_appraisal,mileage_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
price,1.000000,0.350658,NaN,0.482952,0.518392,-0.443925,-0.531439,0.658333,0.536105,0.237564,-0.206117,0.222274,-0.182490,-0.208387,0.326307,0.232716,-0.003977
appraisal_offer,0.350658,1.000000,NaN,0.153745,0.201626,-0.161833,-0.183276,0.261141,0.193339,0.729878,-0.744677,0.206515,-0.133705,-0.177635,0.430583,0.207814,-0.018222
online_appraisal_flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_year,0.482952,0.153745,NaN,1.000000,-0.069913,0.107694,0.056898,0.008616,-0.070012,0.190023,-0.154355,0.000590,0.006878,0.007664,0.057439,0.011661,-0.015320
cylinders,0.518392,0.201626,NaN,-0.069913,1.000000,-0.656309,-0.668066,0.836533,0.739248,0.095922,-0.076896,0.255701,-0.212264,-0.226789,0.292111,0.251079,-0.007036
mpg_city,-0.443925,-0.161833,NaN,0.107694,-0.656309,1.000000,0.912183,-0.736930,-0.715824,-0.091420,0.076690,-0.187252,0.213351,0.213920,-0.234466,-0.202165,0.018844
mpg_highway,-0.531439,-0.183276,NaN,0.056898,-0.668066,0.912183,1.000000,-0.727738,-0.748404,-0.105087,0.091933,-0.193807,0.211048,0.233831,-0.241206,-0.213274,0.017850
horsepower,0.658333,0.261141,NaN,0.008616,0.836533,-0.736930,-0.727738,1.000000,0.747853,0.147800,-0.128464,0.268385,-0.231155,-0.237975,0.346614,0.264359,-0.006888
fuel_capacity,0.536105,0.193339,NaN,-0.070012,0.739248,-0.715824,-0.748404,0.747853,1.000000,0.090803,-0.063119,0.249103,-0.226564,-0.248209,0.286185,0.282889,-0.008919
model_year_appraisal,0.237564,0.729878,NaN,0.190023,0.095922,-0.091420,-0.105087,0.147800,0.090803,1.000000,-0.766964,-0.183154,0.181333,0.208119,0.066199,-0.140446,-0.028629


In [201]:
df2.corr()

/var/folders/pk/vjdnd39913sgqxdcnpt9gxc00000gn/T/ipykernel_31610/1778878097.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df2.corr()


,price,appraisal_offer,online_appraisal_flag,model_year,cylinders,mpg_city,mpg_highway,horsepower,fuel_capacity,model_year_appraisal,mileage_appraisal,cylinders_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,market
price,1.000000,0.401309,NaN,0.482557,0.527183,-0.467513,-0.548120,0.672269,0.555110,0.303141,-0.257428,0.209645,-0.179267,-0.196591,0.351435,0.237904,0.004770
appraisal_offer,0.401309,1.000000,NaN,0.189117,0.221633,-0.187334,-0.213540,0.293304,0.220693,0.716267,-0.732718,0.143833,-0.084182,-0.113032,0.412224,0.162578,0.008114
online_appraisal_flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_year,0.482557,0.189117,NaN,1.000000,-0.048901,0.085352,0.033620,0.032120,-0.046388,0.222860,-0.181185,-0.015116,0.010581,0.019447,0.070903,0.011617,-0.018088
cylinders,0.527183,0.221633,NaN,-0.048901,1.000000,-0.658816,-0.666965,0.831166,0.736850,0.125554,-0.098256,0.260389,-0.216514,-0.232460,0.309226,0.266652,0.000204
mpg_city,-0.467513,-0.187334,NaN,0.085352,-0.658816,1.000000,0.914808,-0.750448,-0.729643,-0.127460,0.102560,-0.212368,0.231638,0.233197,-0.270261,-0.232681,0.015543
mpg_highway,-0.548120,-0.213540,NaN,0.033620,-0.666965,0.914808,1.000000,-0.738245,-0.754516,-0.144144,0.121688,-0.216848,0.227273,0.250226,-0.275853,-0.243152,0.011803
horsepower,0.672269,0.293304,NaN,0.032120,0.831166,-0.750448,-0.738245,1.000000,0.769678,0.195273,-0.161079,0.273821,-0.237752,-0.245102,0.370961,0.285557,0.004857
fuel_capacity,0.555110,0.220693,NaN,-0.046388,0.736850,-0.729643,-0.754516,0.769678,1.000000,0.128700,-0.093090,0.264955,-0.239849,-0.261051,0.313590,0.308267,-0.002325
model_year_appraisal,0.303141,0.716267,NaN,0.222860,0.125554,-0.127460,-0.144144,0.195273,0.128700,1.000000,-0.722235,-0.208578,0.191302,0.241762,0.114333,-0.139468,-0.014821
